In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import regexp_tokenize
import collections
import math
import os
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
from collections import OrderedDict
#nltk.download('stopwords')

In [2]:
# Folder Path
path = "files"
files = [] 
# Change the directory
os.chdir(path)
# iterate through all file
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".txt"):
        files.append(file)

In [3]:
print(files)

['d0.txt', 'd1.txt', 'd2.txt', 'd3.txt', 'd4.txt', 'd5.txt', 'd6.txt', 'd7.txt', 'd8.txt', 'd9.txt']


In [4]:
allTokens = [] #collecting all the terms from the files
stop_words = set(stopwords.words("english")) #to select all important stop words in English language
stop_words.remove("to")
stop_words.remove("in")
stop_words.remove("where")

In [5]:
def readFromFile(path):
    file = open(path, "r+")
    content = file.read()
    file.close()
    return content
    
def Tokenization(theText): #this function remove punctiuations, stop words, make all words in lower case
    word_tokens = word_tokenize(theText) #built-in fn
    for word in word_tokens:
        if word not in string.punctuation and word not in stop_words:
            allTokens.append(word)

def getTerms(): #get all distinct words
    unique = [] 
    for i in allTokens:
        if not i in unique:
            unique.append(i)
    return unique

def getDocs(term): #get all docs that contain a spicific term
    docs = []
    if term in dict:
        for i in dict[term]:
            docs.append(i)
    return docs

def list_duplicates_of(List, item): #get locations of appearence of the term in each document
    start_at = -1
    locs = []
    while True:
        try:
            loc = List.index(item, start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

def positionalIndex(): #the dictionary of the positional index
    dict = {}
    for w in terms:
        for filename in files:
            str = readFromFile(filename)
            str = str.lower()
            list = word_tokenize(str)
            if w in list:
                if w not in dict:
                    dict[w] = {}
                dict[w][filename] = list_duplicates_of(list, w)
    return dict

def printIndex(): #printing the positional index in a readable format
    for term in dict:
        print('<', term, ',', len(dict[term]), ';')
        for doc in dict[term]:
            print("    ", doc,': in index =>', dict[term][doc])
        print('>')

def rawTF(): #calculate (raw-tf) which is number of occurence of each term in each document
    raw = {}
    for term in terms:
        for filename in files:
            str = readFromFile(filename)
            str = str.lower()
            list = word_tokenize(str) 
            if term not in raw:
                raw[term] = {}
            if term in list:
                raw[term][filename] = list.count(term)
            else:
                raw[term][filename] = 0
    return raw

def computeTF(): #dictionary fo tf of each term
    tfDict = {}
    for w in terms:
        for filename in files:
            if w not in tfDict:
                tfDict[w] = {}
            if raw[w][filename] > 0:
                tfDict[w][filename] = 1 + float(math.log10(raw[w][filename]))
            else:
                tfDict[w][filename] = 0.0
    return tfDict

def computeIDF(): #dictionary fo idf of each term
    idfDict = {}
    for w in terms:
        idfDict[w] = float(math.log10( (len(files)) / (len(getDocs(w)) )))
    return idfDict

def tf_idf_matrix(): #dictionary fo the tf-idf matrix
    matrix = {}
    for w in terms:
        for filename in files:
            if w not in matrix:
                matrix[w] = {}
            matrix[w][filename] = tf[w][filename] * idf[w]
    return matrix

def cosine_similarity(vec1, vec2): #multiply 2 unit vectors
    cosine = 0
    for i in range(len(vec1)):
        cosine += vec1[i] * vec2[i]
    return cosine

def queryVector(query_tokens): #to get the unit vector of the query
    vector = []
    queryLength = 0
    for term in terms:
        if term in query_tokens:#"angels fools where angels"
            tf_idf = (1 + float(math.log10(query_tokens.count(term)))) * idf[term]
        else:
            tf_idf = 0.0
        vector.append(tf_idf) #append tf-idf value for each term in the query to a vector with the same size of distinct terms
    
    for i in range(len(vector)):
        queryLength += vector[i] * vector[i]
    queryLength = math.sqrt(queryLength)
    
    if queryLength != 0:
        for i in range(len(vector)):
            vector[i] = vector[i] / float(queryLength)
    return vector

def docVector(doc): #to get the vector of the doc from the tf-idf matrix (before normalization)
    vector = []
    for term in terms:
        vector.append(matrix[term][doc])
    return vector

def normVector(doc): #to get the vector of the doc from the normalized tf-idf matrix (after normalization)
    vector = []
    for term in terms:
        vector.append(normalized[term][doc])
    return vector

def docsLength(): #get the length of each document and store them in a dictionary
    length = {}
    for file in files:
        vec = docVector(file)
        docLength = 0
        for i in range(len(vec)):
            docLength += vec[i] * vec[i]
        docLength = math.sqrt(docLength)
        length[file] = docLength 
    return length

def normalized_tf_idf():
    norm = {}
    for w in terms:
        for filename in files:
            if w not in norm:
                norm[w] = {}
            norm[w][filename] = matrix[w][filename] / docs_length[filename]
    return norm

def tokenizeQuery(query):
    query = query.lower()
    queryList = word_tokenize(query)
    query_tokens = []
    for word in queryList:
        if word not in string.punctuation and word not in stop_words:
            query_tokens.append(word)
    return query_tokens

def docsIntersection(query_tokens):
    term_docs = [] #contains array of documents that contain each term
    for w in query_tokens:
        term_docs.append(getDocs(w))
        
    matchDocs = [] #contains all matched documents 
    all = []
    for i in term_docs:
        for j in i:
            all.append(j)
    for i in all:
        if all.count(i) == len(query_tokens):
            if i not in matchDocs:
                matchDocs.append(i)
    return matchDocs #contain all matched documents

def searchingAlgo(matchDocs):
    match_ind = [] #contains arrays of locations of each term in each doc of matched documents
    for doc in matchDocs:
        for i in query_tokens:
            match_ind.append(dict[i][doc]) #[DocName --> arrays equal to number of terms in query]
    
    all_matched = [] #contains all matched positions for the query in each document
    last_matched = [] #contains the compination between the top two lists
    while len(all_matched) != len(matchDocs):
        for m in range(len(query_tokens) - 1):
            #take the first 2 posting lists of locations
            x = match_ind[0]
            y = match_ind[1]
            p1 = 0
            p2 = 0
            
            while True: #checking if they are beside each other
                if p1 == len(x) or p2 == len(y):
                    break
                if x[p1] + 1 == y[p2]:
                    last_matched.append(y[p2])
                    p1 = p1 + 1
                    p2 = p2 + 1
                elif x[p1] < y[p2]:
                    p1 = p1 + 1
                else:
                    p2 = p2 + 1

            match_ind.pop(0)
            match_ind.pop(0)
            #insert the new matched positions to compare it with the next term's positions
            match_ind.insert(0, last_matched)
            last_matched = []
            
        all_matched.append(match_ind[0])
        match_ind.pop(0)
        
    return all_matched

def getResult(all_matched):
    result = []
    for i in range(len(matchDocs)):
        if len(all_matched[i]) != 0:
            result.append(matchDocs[i])
    return result

def ranking(query_tokens, result): #to rank the final documents
    qv = queryVector(query_tokens) #get the unit vector of the query  
    answer = {} 
    for doc in result: 
        #calculate the cosine similarity between the query vector and each document vector in the result set 
        cosine = cosine_similarity(qv, normVector(doc))
        answer[doc] = cosine
    #to sort the score deseanding
    return {k: v for k, v in sorted(answer.items(), key=lambda item: item[1], reverse = True)}

def finalResult(): #printing in order of the score  of each document
    for i in ranked:
        print(i)
        

In [6]:
for filename in files:
    str = readFromFile(filename)
    str = str.lower()
    Tokenization(str)

In [7]:
print(allTokens) #testing

['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser', 'antony', 'brutus', 'caeser', 'calpurnia', 'mercy', 'worser', 'brutus', 'caeser', 'mercy', 'worser', 'caeser', 'mercy', 'worser', 'antony', 'caeser', 'mercy', 'angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where', 'angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where', 'angels', 'fools', 'in', 'rush', 'to', 'tread', 'where', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']


In [8]:
terms = getTerms()
terms.sort()
print(terms)

['angels', 'antony', 'brutus', 'caeser', 'calpurnia', 'cleopatra', 'fear', 'fools', 'in', 'mercy', 'rush', 'to', 'tread', 'where', 'worser']


In [9]:
dict = positionalIndex()
print(dict)

{'angels': {'d6.txt': [0], 'd7.txt': [0], 'd8.txt': [0]}, 'antony': {'d0.txt': [0], 'd1.txt': [0], 'd5.txt': [0]}, 'brutus': {'d0.txt': [1], 'd1.txt': [1], 'd3.txt': [0]}, 'caeser': {'d0.txt': [2], 'd1.txt': [2], 'd3.txt': [1], 'd4.txt': [0], 'd5.txt': [1]}, 'calpurnia': {'d1.txt': [3]}, 'cleopatra': {'d0.txt': [3]}, 'fear': {'d6.txt': [2], 'd7.txt': [2], 'd9.txt': [1]}, 'fools': {'d6.txt': [1], 'd7.txt': [1], 'd8.txt': [1], 'd9.txt': [0]}, 'in': {'d6.txt': [3], 'd7.txt': [3], 'd8.txt': [2], 'd9.txt': [2]}, 'mercy': {'d0.txt': [4], 'd2.txt': [0], 'd3.txt': [2], 'd4.txt': [1], 'd5.txt': [2]}, 'rush': {'d6.txt': [4], 'd7.txt': [4], 'd8.txt': [3], 'd9.txt': [3]}, 'to': {'d6.txt': [5], 'd7.txt': [5], 'd8.txt': [4], 'd9.txt': [4]}, 'tread': {'d6.txt': [6], 'd7.txt': [6], 'd8.txt': [5], 'd9.txt': [5]}, 'where': {'d6.txt': [7], 'd7.txt': [7], 'd8.txt': [6], 'd9.txt': [6]}, 'worser': {'d0.txt': [5], 'd2.txt': [1], 'd3.txt': [3], 'd4.txt': [2]}}


In [10]:
printIndex() #in a readable format

< angels , 3 ;
     d6.txt : in index => [0]
     d7.txt : in index => [0]
     d8.txt : in index => [0]
>
< antony , 3 ;
     d0.txt : in index => [0]
     d1.txt : in index => [0]
     d5.txt : in index => [0]
>
< brutus , 3 ;
     d0.txt : in index => [1]
     d1.txt : in index => [1]
     d3.txt : in index => [0]
>
< caeser , 5 ;
     d0.txt : in index => [2]
     d1.txt : in index => [2]
     d3.txt : in index => [1]
     d4.txt : in index => [0]
     d5.txt : in index => [1]
>
< calpurnia , 1 ;
     d1.txt : in index => [3]
>
< cleopatra , 1 ;
     d0.txt : in index => [3]
>
< fear , 3 ;
     d6.txt : in index => [2]
     d7.txt : in index => [2]
     d9.txt : in index => [1]
>
< fools , 4 ;
     d6.txt : in index => [1]
     d7.txt : in index => [1]
     d8.txt : in index => [1]
     d9.txt : in index => [0]
>
< in , 4 ;
     d6.txt : in index => [3]
     d7.txt : in index => [3]
     d8.txt : in index => [2]
     d9.txt : in index => [2]
>
< mercy , 5 ;
     d0.txt : in index =

In [11]:
raw = rawTF()
pd.DataFrame(raw) #calculate (raw-tf) which is number of occurence of each term in each document

,angels,antony,brutus,caeser,calpurnia,cleopatra,fear,fools,in,mercy,rush,to,tread,where,worser
d0.txt,0,1,1,1,0,1,0,0,0,1,0,0,0,0,1
d1.txt,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0
d2.txt,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
d3.txt,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1
d4.txt,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1
d5.txt,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0
d6.txt,1,0,0,0,0,0,1,1,1,0,1,1,1,1,0
d7.txt,1,0,0,0,0,0,1,1,1,0,1,1,1,1,0
d8.txt,1,0,0,0,0,0,0,1,1,0,1,1,1,1,0
d9.txt,0,0,0,0,0,0,1,1,1,0,1,1,1,1,0


In [12]:
tf = computeTF()
pd.DataFrame(tf)

,angels,antony,brutus,caeser,calpurnia,cleopatra,fear,fools,in,mercy,rush,to,tread,where,worser
d0.txt,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
d1.txt,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d2.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
d3.txt,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
d4.txt,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
d5.txt,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
d6.txt,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
d7.txt,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
d8.txt,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
d9.txt,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0


In [13]:
idf = computeIDF()
pd.DataFrame(idf, index = ["IDF"])

,angels,antony,brutus,caeser,calpurnia,cleopatra,fear,fools,in,mercy,rush,to,tread,where,worser
IDF,0.522879,0.522879,0.522879,0.30103,1.0,1.0,0.522879,0.39794,0.39794,0.30103,0.39794,0.39794,0.39794,0.39794,0.39794


In [14]:
matrix = tf_idf_matrix()
pd.DataFrame(matrix) #before normalization

,angels,antony,brutus,caeser,calpurnia,cleopatra,fear,fools,in,mercy,rush,to,tread,where,worser
d0.txt,0.000000,0.522879,0.522879,0.30103,0.0,1.0,0.000000,0.00000,0.00000,0.30103,0.00000,0.00000,0.00000,0.00000,0.39794
d1.txt,0.000000,0.522879,0.522879,0.30103,1.0,0.0,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
d2.txt,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.00000,0.00000,0.30103,0.00000,0.00000,0.00000,0.00000,0.39794
d3.txt,0.000000,0.000000,0.522879,0.30103,0.0,0.0,0.000000,0.00000,0.00000,0.30103,0.00000,0.00000,0.00000,0.00000,0.39794
d4.txt,0.000000,0.000000,0.000000,0.30103,0.0,0.0,0.000000,0.00000,0.00000,0.30103,0.00000,0.00000,0.00000,0.00000,0.39794
d5.txt,0.000000,0.522879,0.000000,0.30103,0.0,0.0,0.000000,0.00000,0.00000,0.30103,0.00000,0.00000,0.00000,0.00000,0.00000
d6.txt,0.522879,0.000000,0.000000,0.00000,0.0,0.0,0.522879,0.39794,0.39794,0.00000,0.39794,0.39794,0.39794,0.39794,0.00000
d7.txt,0.522879,0.000000,0.000000,0.00000,0.0,0.0,0.522879,0.39794,0.39794,0.00000,0.39794,0.39794,0.39794,0.39794,0.00000
d8.txt,0.522879,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.39794,0.39794,0.00000,0.39794,0.39794,0.39794,0.39794,0.00000
d9.txt,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.522879,0.39794,0.39794,0.00000,0.39794,0.39794,0.39794,0.39794,0.00000


In [15]:
docs_length = docsLength()
pd.DataFrame(docs_length, index=["length"])

,d0.txt,d1.txt,d2.txt,d3.txt,d4.txt,d5.txt,d6.txt,d7.txt,d8.txt,d9.txt
length,1.373462,1.279618,0.498974,0.782941,0.582747,0.67427,1.223496,1.223496,1.106137,1.106137


In [16]:
normalized = normalized_tf_idf()
pd.DataFrame(normalized) #after normalization (unit vector to each document)

,angels,antony,brutus,caeser,calpurnia,cleopatra,fear,fools,in,mercy,rush,to,tread,where,worser
d0.txt,0.000000,0.380701,0.380701,0.219176,0.000000,0.728087,0.000000,0.000000,0.000000,0.219176,0.000000,0.000000,0.000000,0.000000,0.289735
d1.txt,0.000000,0.408621,0.408621,0.235250,0.781483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d2.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.603298,0.000000,0.000000,0.000000,0.000000,0.797516
d3.txt,0.000000,0.000000,0.667839,0.384486,0.000000,0.000000,0.000000,0.000000,0.000000,0.384486,0.000000,0.000000,0.000000,0.000000,0.508263
d4.txt,0.000000,0.000000,0.000000,0.516570,0.000000,0.000000,0.000000,0.000000,0.000000,0.516570,0.000000,0.000000,0.000000,0.000000,0.682869
d5.txt,0.000000,0.775474,0.000000,0.446453,0.000000,0.000000,0.000000,0.000000,0.000000,0.446453,0.000000,0.000000,0.000000,0.000000,0.000000
d6.txt,0.427365,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.325248,0.325248,0.000000,0.325248,0.325248,0.325248,0.325248,0.000000
d7.txt,0.427365,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.325248,0.325248,0.000000,0.325248,0.325248,0.325248,0.325248,0.000000
d8.txt,0.472707,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.359756,0.359756,0.000000,0.359756,0.359756,0.359756,0.359756,0.000000
d9.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.472707,0.359756,0.359756,0.000000,0.359756,0.359756,0.359756,0.359756,0.000000


# The main()

In [19]:
query = input("Enter query here --> ") 
query_tokens = tokenizeQuery(query) 
matchDocs = docsIntersection(query_tokens)
all_matched = searchingAlgo(matchDocs) 
result = getResult(all_matched)

print("All matched documents that contain the terms in general --> ", matchDocs)
print("The tokens --> ", query_tokens) 
print("The phrase query exists in --> ", result) 

ranked = ranking(query_tokens, result)
print("Scores of each document --> ", ranked)
print("==================================")
print("The returned documents: ")
finalResult()

Enter query here --> fools fear in
All matched documents that contain the terms in general -->  ['d6.txt', 'd7.txt', 'd9.txt']
The tokens -->  ['fools', 'fear', 'in']
The phrase query exists in -->  ['d6.txt', 'd7.txt', 'd9.txt']
Scores of each document -->  {'d9.txt': 0.6944790999587873, 'd6.txt': 0.6278642317939398, 'd7.txt': 0.6278642317939398}
The returned documents: 
d9.txt
d6.txt
d7.txt
